In [ ]:
import pandas as pd
import numpy as np

import uuid
import pickle
import ast
import os

from Bio import AlignIO
from Bio import SeqIO
from Bio import AlignIO

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter
from collections import Counter

# Calculate and map the Codon rarity at each position of a multiple sequence alignment

### Function to calculate the codon rarity at each position of a multiple sequence alignment:
CRS = Codon Rarity Score, AA = Amino Acid, occ = Occurence, f_c = Frequency of codon, len = Length, aln = Alignment, gaps = Gaps, n_aln = Number of sequences in the alignment
$$ CRS_{column}= {\sum \limits _{AA_{AA}} ^{n_{aln}}(\sum \limits _{occ=1} ^{n_{aln}} {AA_{occ}} * f_c) \over {len_{total}(alignment)} - (gaps)} $$
$$ f_c = { \sum n_c \over \sum n_{AA} } * { 1 \over n_{cAA} } $$

### Set data to analyse

In [ ]:
savefig = True # define if figures should be saved

# assign the paths to the files need for the analysis
alignment_file = "/Example/examples_family/example_fam1_aln.fasta" 
nustrudb =  "/Example/examples_family/example_fam1_nustrudb.csv"
output_path = "/Example/examples_family/"

# get a working name from the input file (e.g. protein family name) to use in the output files
working_name = os.path.basename(alignment_file)

nt_fasta_file = None  # nucleotide sequences in fasta format (optional, so no extra file will be created)

# for the temporary created files, like the nucleotide sequences, create a unique job_id
job_id = uuid.uuid4()

# read the nucleotide structure database as csv
nustrudb = pd.read_csv(nustrudb)

# read the protein fasta alignment and nucleotide fasta sequences if provided
protein_alignment = AlignIO.read(alignment_file, "fasta")
if nt_fasta_file is not None:
    nucleotide_sequences = SeqIO.parse(nt_fasta_file, "fasta")
else:
    # else create a temporary file with the nucleotide sequences corresponding to the protein sequences
    for record in protein_alignment:
        try:
            # get the nucleotide id and sequence from the nustrudb
            # and write the id and sequence to a temporary file in the defined output path
            nucleotide_id = nustrudb.loc[nustrudb["primary_id"] == record.id]["nucleotide_id"].values[0]
            nucleotide_sequence = nustrudb.loc[nustrudb["primary_id"] == record.id]["nucleotide_sequence"].values[0]
            with open(f"{output_path}/{job_id}_nucleotide_sequences.fasta", "a") as f:
                f.write(f">{nucleotide_id}\n{nucleotide_sequence}\n")
        except:
            continue
    
    # then read the temporary created nucleotide sequences
    nucleotide_sequences = SeqIO.parse(f"{output_path}/{job_id}_nucleotide_sequences.fasta", "fasta")

### Define functions

In [ ]:
def fasta_to_array(fasta, align_to=None, codon=False):
    """Converts a fasta file to a numpy array"""
    # set empty lists to store the sequences and ids
    all_seqs = []
    all_ids = []
    
    if codon:
        # if the sequences are nucleotide sequences and codons should be used
        # then split the sequences into codons for each sequence in the fasta file
        for (ind,record) in enumerate(fasta):
            all_seqs.append(list(str(record.seq)))
            all_seqs[ind] = [''.join(map(str, all_seqs[ind][i:i+3])) for i in range(0, len(all_seqs[ind]), 3)]
            
    else:         
        # else just append the sequences and ids to the lists 
        # this can be done always for protein sequences  
        for record in fasta:
            all_seqs.append(list(str(record.seq)))
            all_ids.append(record.id)
        
        # convert the sequences to a numpy array
        all_seqs = np.array(all_seqs)

    if align_to is not None:
        # if an alignment is provided, then insert gaps in the sequences where gaps are in the alignment
        # get the gap positions in the alignment and create a numpy array as a mask
        gap_indeces = np.where(align_to == '-')
        
        # loop through the gap positions and insert a gap in the sequences
        for gap_index in zip(gap_indeces[0], gap_indeces[1]):
            # insert the gap index at the correpsonding position in the sequences
            # since we align the codons we use 3 dashes for a gap to show nucleotide triplets
            all_seqs[gap_index[0]].insert(gap_index[1], '---')
        
        # convert the aligned sequences with gaps to a numpy array
        all_seqs = np.array(all_seqs)
        # deleting stop codons as no protein assigned to them
        all_seqs = np.delete(all_seqs, -1, axis=1)
    
    # reshape the ids to a numpy array according to the number of sequences (one column)
    all_ids = np.array(all_ids).reshape(len(all_ids), 1)
    
    return all_seqs

def filter_columns_by_gap_threshold(seq_arr, threshold=0.5):
    """Filter columns in a sequence array based on a gap threshold"""
    # get the number of rows in the sequence array
    n_rows = seq_arr.shape[0]
    # create empty lists to store the valid and deleted columns
    valid_columns = []
    deleted_columns = []
    
    # Iterate through each column index
    for col_index in range(seq_arr.shape[1]):
        gap_count = np.sum(seq_arr[:, col_index] == 0)
        gap_percentage = gap_count / n_rows
        
        # Append column index based on gap percentage
        if gap_percentage <= threshold:
            valid_columns.append(col_index)
        else:
            deleted_columns.append(col_index)
    
    # get all rows and only the valid columns where the gap percentage is below the threshold
    filtered_seq_arr = seq_arr[:, valid_columns]
    
    return filtered_seq_arr, deleted_columns

def cub_msa_table(prot_seq_arr=None, cod_seq_arr=None):
    """Create a codon usage bias table for the multiple sequence alignment"""
    cub_table = {
    # '*': {'TAA': None, 'TAG': None, 'TGA': None}, ignoring stop codons
    'A': {'GCA': None, 'GCC': None, 'GCG': None, 'GCT': None},
    'C': {'TGC': None, 'TGT': None},
    'D': {'GAC': None, 'GAT': None},
    'E': {'GAA': None, 'GAG': None},
    'F': {'TTC': None, 'TTT': None},
    'G': {'GGA': None, 'GGC': None, 'GGG': None, 'GGT': None},
    'H': {'CAC': None, 'CAT': None},
    'I': {'ATA': None, 'ATC': None, 'ATT': None},
    'K': {'AAA': None, 'AAG': None},
    'L': {'CTA': None, 'CTC': None, 'CTG': None, 'CTT': None, 'TTA': None, 'TTG': None},
    'M': {'ATG': None},
    'N': {'AAC': None, 'AAT': None},
    'P': {'CCA': None, 'CCC': None, 'CCG': None, 'CCT': None},
    'Q': {'CAA': None, 'CAG': None},
    'R': {'AGA': None, 'AGG': None, 'CGA': None, 'CGC': None, 'CGG': None, 'CGT': None},
    'S': {'AGC': None, 'AGT': None, 'TCA': None, 'TCC': None, 'TCG': None, 'TCT': None},
    'T': {'ACA': None, 'ACC': None, 'ACG': None, 'ACT': None},
    'V': {'GTA': None, 'GTC': None, 'GTG': None, 'GTT': None},
    'W': {'TGG': None},
    'Y': {'TAC': None, 'TAT': None}}
    
    for aa in cub_table.keys():
        # loop through the amino acids and codons in the cub table and calculate the frequency
        # n_AA is the number of occurences of the amino acid in the protein sequence
        n_AA = np.count_nonzero(prot_seq_arr == aa)
                
        # nc_AA is the number of synonyomous codons for the amino acid
        nc_AA = len(cub_table[aa].keys())
        
        for codon in cub_table[aa].keys():
            # now for each codon a frequency based on the alignment is calculated
            # nc is the number of occurences of the codon in the in the sequences
            nc = np.count_nonzero(cod_seq_arr == codon)
            
            # fc is the frequency of the codon in the alignment
            # it is calculated by dividing the number of occurences of the codon by the number of occurences of the amino acid
            # then its normalized by the number of synonymous codons
            fc =(nc / n_AA) * 1/nc_AA

            # round the frequency to 6 decimal places and create an cub table with the frequencies based on the msa
            cub_table[aa][codon] = round(fc,6)
        
    return cub_table

def map_rarity(protein_alignment, nustrudb, cu_table):
    """Map the rarity of the codons to the protein alignment"""
    codon_position_start = 0
    
    # calculate amino acid occurence in the alignment
    # and create a dictionary with the amino acid and its occurence
    unique_aa, aa_counts = np.unique(protein_alignment, return_counts=True)
    aa_counts = dict(zip(unique_aa, aa_counts))
    
    # create an empty matrix to store the rarity values
    alignment_value_matrix = np.zeros((len(protein_alignment), len(protein_alignment[0])))
    seq_name = [seq.id for seq in protein_alignment] # get the sequence names or ids from the alignment
    seq_pos = [i for i in range(len(protein_alignment[0]))] # get the sequence positions (with gaps)

    # keep track of the position in the alignment for each sequence
    # this is needed to adjust the position based on the gaps in the alignment
    # so keeps track of the gaps in the alignment
    pos_count_dict = {seq_name[i]: 0 for i in range(len(seq_name))}
    
    # loop over the sequences in the alignment
    for position in range(len(protein_alignment[0])):
        # loop over the amino acids and sequences in the alignment
        for i, (aa, seq) in enumerate(zip(protein_alignment[:,position],seq_name)):
            # if the amino acid is a gap, then set the rarity to 0
            # else calculate the rarity based on the codon usage table
            if aa == '-':
                # add gaps to the alignment value matrix
                alignment_value_matrix[i, position] = 0 # gaps have no information
                pos_count_dict[seq] += 1 # count the gaps for each sequence
            else:
                # adjust the position in the nucleotide sequence based on the gaps in the alignment
                prot_position = pos_count_dict[seq]
                position_adj = position - prot_position 
                
                # get the nucleotide sequence from the nustrudb
                sequence = nustrudb[nustrudb["primary_id"] == seq]["nucleotide_sequence"].values[0]
                # set the rarity value based on the codon usage table
                # since the codon usage table is based on codons, we need to adjust the position by 3
                # the values are then stored in the equivalent position in the alignment value matrix
                alignment_value_matrix[i, position] = cu_table[aa][sequence[position_adj*3:position_adj*3+3].upper()]

    # calculate the mean rarity of the residues in the alignment
    # divide the sum of the rarity values in a column by the number of sequences
    # subtract the number of gaps in the column from the number of sequences
    # vertical mean of the rarity values in a column (column mean)  
    residue_mean = []
    for i, col_mean in enumerate(np.sum(alignment_value_matrix, axis=0)):
        residue_mean.append(col_mean / (len(seq_name) - np.count_nonzero(alignment_value_matrix[:, i] == 0)))

    # calculate the max rarity of the residues in the alignment
    # vertical max of the rarity values in a column
    residue_max = []
    for col_max in np.max(alignment_value_matrix, axis=0):
        residue_max.append(col_max)
    
    return alignment_value_matrix, seq_name, seq_pos, residue_mean, residue_max

## Transform alignment and sequences to arrays and align the gaps if necessary

In [ ]:
# convert the protein and nucleotide sequences to numpy arrays
# nucleotide sequences are aligned to the protein sequences and split into codons
all_seqs_protein = fasta_to_array(protein_alignment)
all_seqs_nt = fasta_to_array(fasta=nucleotide_sequences, align_to=all_seqs_protein, codon=True)

## Calculate the codon frequency for each amino acid based on the alignment

#### $$ f_c = { \sum n_c \over \sum n_{AA} } * { 1 \over n_{cAA} } $$

In [ ]:
# calculate the rarity of the codons in the alignment based on the formula
cub_msa_table = cub_msa_table(prot_seq_arr=all_seqs_protein, cod_seq_arr=all_seqs_nt)

In [ ]:
# create a dictionary with the rarity values based on the alignment
# this is needed for other scripts like the fold class analysis
merged_dict = {key: value for codon in cub_msa_table.values() for key, value in codon.items()}

# store the rarity values in a pickle file which can be used in other scripts
with open(f"{output_path}/{working_name}_cub_msa_table.pkl", "wb") as f:
    pickle.dump(merged_dict, f)

## Calculate the Codon Rarity Score for each position of the alignment

#### $$ CRS_{column}= {\sum \limits _{AA_{AA}} ^{n_{aln}}(\sum \limits _{occ=1} ^{n_{aln}} {AA_{occ}} * f_c) \over {len_{total}(alignment)} - (gaps)} $$

In [ ]:
# map the rarity values to the alignment and create a numpy array based on the positions
# calculate the mean rarity of the residues and columns in the alignment
# also get the sequence names and positions for each row in the alignment
alignment_value_matrix, seq_name, seq_pos, residue_mean, residue_max = map_rarity(protein_alignment, nustrudb, cub_msa_table)

# sort the CRS matrix and flip it to get the sorted values
# just for visualization purposes later
sorted_alignment_value_matrix = np.sort(alignment_value_matrix, axis=0)
sorted_alignment_value_matrix = np.flip(sorted_alignment_value_matrix, axis=0)

In [ ]:
# filter the columns in the alignment based on a gap threshold, here 50%
# save the deleted columns to a list
alignment_value_matrix_filtered,deleted_columns = filter_columns_by_gap_threshold(alignment_value_matrix)
sorted_alignment_value_matrix_filtered,deleted_columns = filter_columns_by_gap_threshold(sorted_alignment_value_matrix)

In [ ]:
# use the deleted columns to filter the sequence positions, residue mean and residue max
# this aligns all values to the filtered alignment matrix
for index in sorted(deleted_columns, reverse=True):
    del seq_pos[index]
    del residue_max[index]
    del residue_mean[index]

In [ ]:
# get the sequence positions for the filtered alignment matrix
# the old seq_pos can be also used
seq_pos_new = [i for i in range(len(residue_max))]

## Visualize the Codon Rarity Score for each position of the alignment in MSA

##### The window is set to 10 positions, but it can be changed to any value

In [ ]:
# create a pandas dataframe with the sequence positions and the residue mean
df = pd.DataFrame({
    'x': seq_pos,
    'y': residue_mean
})

# Apply a Savitzky-Golay filter to the residue mean values
# this is done to smooth the curve and remove noise
# the window size is 10 and the polynomial order is 3
df['smoothed_y'] = savgol_filter(df['y'], 10, 3)

In [ ]:
# plot the codon rarity matrix as a heatmap and the CRS mean per column/position as a line plot
# used plotly for the heatmap and line plot as it is interactive and subplots are easier to create
fig1 = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig1.add_trace((go.Heatmap(z=alignment_value_matrix_filtered, y=seq_name, colorscale="blues", showlegend=False)), row=3, col=1)
fig1.add_trace((go.Heatmap(z=sorted_alignment_value_matrix_filtered, y=seq_name, colorscale="blues")), row=2, col=1)
fig1.add_trace((go.Scatter(x=seq_pos_new, y=df['smoothed_y'], mode="lines", line=dict(color="navy"))), row=1, col=1)
fig1['layout']['yaxis1']['visible']=True
fig1['layout']['yaxis2']['visible']=False
fig1['layout']['yaxis3']['visible']=False

if savefig:
    fig1.write_image(f"{output_path}/{working_name}_codon_rarity_heatmap.png", scale=2)
    fig1.write_html(f"{output_path}/{working_name}_codon_rarity_heatmap.html")

## Calculate the correlation between the smoothed Y and the secondary structure frequencies as a log odds ratio (rare codons and secondary structure)

### Define new functions for the analysis

In [ ]:
def get_secondary_structure(seq_names, aln_prot_arr, nustrudb):
    """Create a list with the aligned secondary structure arrays"""
    aligned_secstru_list = []

    for seq in seq_names:
        # Retrieve the secondary structure dictionary for the current sequence
        secstru_dict = nustrudb[nustrudb["primary_id"] == seq]["secondary_structure"].values[0]
        
        secstru_dict = ast.literal_eval(secstru_dict)
        # Convert the dictionary to a list that aligns with the protein sequence length
        max_pos = max(secstru_dict.keys())
        secstru_list = ['-' for _ in range(max_pos)]
        for pos, ss in secstru_dict.items():
            secstru_list[pos - 1] = ss  # Adjust for 1-based to 0-based indexing
        
        # Align the secondary structure list with the protein alignment
        aligned_secstru = []
        secstru_idx = 0
        
        # previously gap postions were filtered out with more than 50% gaps
        # however since there are still gaps present in the alignment, it is necessary to adjust the secondary structure list
        for aa in aln_prot_arr[seq_names.index(seq)]:
            if aa == '-': # if gap, add a gap to the secondary structure list
                aligned_secstru.append('0')
            else: # if not a gap, add the secondary structure element
                aligned_secstru.append(secstru_list[secstru_idx])
                secstru_idx += 1
        
        # append the aligned secondary structure list to the list of lists
        aligned_secstru_list.append(aligned_secstru) 
    
    # convert the list of lists to a numpy array
    # this makes it easier to work with the data later on and to plot the data
    return np.array(aligned_secstru_list)

def secstru_to_numeric(secstru_arr):
    """Convert secondary structure elements to numeric values for plotting"""
    # store the numeric secondary structure arrays in a list
    secstru_numeric = []
    
    for secstru in secstru_arr:
        # loop through each secondary structure element in the array and append the numeric value to a new list
        # the secondary structure elements are aggregated into three categories: helix, sheet, and coil
        numeric_seq = []
        for element in secstru:
            if element == 'H' or element == 'I' or element == 'G':
                numeric_seq.append(1)
            elif element == 'E' or element == 'B':
                numeric_seq.append(2)
            elif element == '-' or element == 'T' or element == 'S':
                numeric_seq.append(3)
            elif element == '0': # previously gaps were assigned to 0 and are just kept as 0
                numeric_seq.append(0)
            else: # if an unknown secondary structure element is present, assign it to 0 (normally not the case)
                numeric_seq.append(0)
        
        # append the numeric secondary structure array to the list of numeric arrays
        secstru_numeric.append(numeric_seq)
        
    # convert the list of numeric arrays to a numpy array
    return np.array(secstru_numeric)    

def calculate_probabilities(secstru_numeric, rarity_matrix, rare_threshold):
    """Calculate the probabilities of rare codons in each secondary structure element"""
    rare_codon_numeric = rarity_matrix < rare_threshold # the definition of rare codons is based on the rarity threshold
    probabilities = [] # list to store the probabilities for each position

    for pos in range(secstru_numeric.shape[1]):
        # loop over each position in the alignment
        prob_pos = {} # probability dictionary for the current position
        
        for ss_type in [1, 2, 3]:  # H, E, and Coil
            # loop over each secondary structure element and calculate the probabilities
            # calculate the joint probability of a secondary structure element and a rare codon at the current position

            
            # calculate the probabilities for each secondary structure element at the current position
            # meaning the independent probability of a secondary structure element
            prob_ss = np.mean(secstru_numeric[:, pos] == ss_type)
            
            # calculate the probability of a rare codon at the current position
            # meaning the independent probability of a rare codon
            prob_rare = np.mean(rare_codon_numeric[:, pos])
            
            # calculate the joint probability of a secondary structure element and a rare codon at the current position
            joint_prob = np.mean((secstru_numeric[:, pos] == ss_type) & rare_codon_numeric[:, pos])

            # store the probabilities in a dictionary
            # for each position, the probabilities for each secondary structure element are stored
            prob_pos[ss_type] = {
                "joint_prob": joint_prob,
                "prob_ss": prob_ss,
                "prob_rare": prob_rare
            }
        # append the probabilities for the current position to the list of probabilities
        probabilities.append(prob_pos)
    
    return probabilities

def calculate_odds_ratios(probabilities):
    """Calculate the odds ratios for each secondary structure element having a rare codon"""
    odds_ratios = []

    # loop over the probabilities for each position
    for prob_pos in probabilities:
        # store the odds ratios for each secondary structure element in a dictionary
        odds_ratios_pos = {}
        for ss_type in [1, 2, 3]:  # H, E, and Coil
            joint_prob = prob_pos[ss_type]["joint_prob"]
            prob_ss = prob_pos[ss_type]["prob_ss"]
            prob_rare = prob_pos[ss_type]["prob_rare"]

            try:
                # calculate the odds ratio for each secondary structure element
                odds_ratio = joint_prob / (prob_ss * prob_rare)
            except:
                odds_ratio = 0 # Handle division by zero

            # assign the odds ratio to the dictionary for the current position and secondary structure element
            odds_ratios_pos[ss_type] = odds_ratio
        # append the odds ratios for the current position to the list of odds ratios
        odds_ratios.append(odds_ratios_pos)
    
    return odds_ratios

def log_scale_odds_ratios(odds_ratios):
    """"Log transform the odds ratios"""
    log_odds_ratios = []
    
    # loop over the odds ratios for each position
    for odds in odds_ratios:
        # store the log odds ratios in a dictionary
        log_odds = {}
        for key, value in odds.items():
            log_odds[key] = np.log(value)
        # append the log odds ratios for the current position to the list of log odds ratios
        # same as before, only this time the odds ratios are log transformed
        log_odds_ratios.append(log_odds)
    return log_odds_ratios

### Plot the log odds ratios

In [ ]:
# get the secondary structure for the protein sequences
aligned_secstru_arr = get_secondary_structure(seq_name, all_seqs_protein, nustrudb)

# convert the secondary structure elements to numeric values
secstru_numeric = secstru_to_numeric(aligned_secstru_arr)

# filter the columns in the alignment based on a gap threshold from previously
secstru_numeric_ng = np.delete(secstru_numeric, deleted_columns, axis=1)

# calculate the probabilities of rare codons in each secondary structure element
# get the log odds of having a rare codon together with a secondary structure element
rare_threshold = 0.3 # Define your threshold for rarity
probabilities = calculate_probabilities(secstru_numeric_ng, alignment_value_matrix, rare_threshold)
odds_ratios = calculate_odds_ratios(probabilities)
    
# log transform the odds ratios
log_odds_ratios = log_scale_odds_ratios(odds_ratios)

# create an array with the positions for the secondary structure elements
positions = np.arange(len(log_odds_ratios))

# create lists of the log odds ratios for each secondary structure element
odds_H = [or_dict[1] for or_dict in log_odds_ratios]
odds_E = [or_dict[2] for or_dict in log_odds_ratios]
odds_C = [or_dict[3] for or_dict in log_odds_ratios]

In [ ]:
# set the bar width for the plot
bar_width = 0.8

fig, ax = plt.subplots(figsize=(14, 8))
plt.style.use('seaborn-v0_8-whitegrid')

ax.bar(positions, odds_H, color='cadetblue', width=bar_width, label='Helix (H)')
ax.bar(positions, odds_E, color='coral', width=bar_width, label='Sheet (E)')
ax.bar(positions, odds_C, color='mediumseagreen', width=bar_width, label='Coil (C)')
# ax.plot(positions, df['smoothed_y'], label='Smoothed Y', linewidth=2, linestyle='--', color='blue')

ax.set_xlim([-5, len(positions)])
ax.set_xlabel('Position')
ax.set_ylabel('Log Odds Ratio')
ax.legend()
ax.set_xticks(positions[::5])

plt.tight_layout()
if savefig:
    plt.savefig(f"{output_path}/{working_name}_log_odds_ratios.png", dpi=600)

# See the consesus secondary structure of the alignment with the Codon Rarity Score

In [ ]:
# get the consensus secondary structure for the protein sequences
consensus_numeric = []

# loop over the positions in the secondary structure numeric array
for pos in range(secstru_numeric_ng.shape[1]):
    # get the secondary structure elements at the current position
    pos_elements = secstru_numeric_ng[:, pos]
    
    # use the Counter class to get the most common secondary structure element at the current position
    most_common = Counter(pos_elements).most_common(1)[0][0]
    # append the most common secondary structure element to the consensus list
    consensus_numeric.append(most_common)

# create a plot with the consensus secondary structure and the smoothed CRS mean
positions = np.arange(len(consensus_numeric))
fig, ax1 = plt.subplots(figsize=(14, 3)) 

colors = {1: 'cadetblue', 2: 'coral', 3: 'mediumseagreen', 0: 'lightgray'}
labels = {1: 'Helix', 2: 'Sheet', 3: 'Coil', 0: 'Gap'}

for pos in positions:
    ax1.bar(pos, 1, color=colors[consensus_numeric[pos]], edgecolor='none', width=1)

ax1.set_xticks(positions[::10]) 
ax1.set_xticklabels(positions[::10] + 1, rotation=90)
ax1.set_yticks([])
ax1.set_xlim([-1, len(consensus_numeric)])
ax1.set_xlabel('Position')
ax1.grid(False)

legend_elements = [plt.Line2D([0], [0], color=colors[i], lw=6, label=labels[i]) for i in [1, 2, 3, 0]]

ax2 = ax1.twinx() 
ax2.plot(seq_pos_new, df['smoothed_y'], color="navy", linewidth=2, linestyle='-')
ax2.tick_params(axis='y', labelcolor="navy")
ax2.legend(bbox_to_anchor=(0.6, 1.3), ncol=5, handles=legend_elements)

plt.tight_layout()
if savefig:
    plt.savefig(f"{output_path}/{working_name}_consensus.png", dpi=600)